# Generación de textos con Deep Learning

Las redes neuronales recurrentes también pueden usarse como modelos generativos. Esto significa que, además de ser utilizados para modelos predictivos (hacer predicciones), pueden aprender las secuencias de un problema y luego generar secuencias plausibles completamente nuevas para el dominio del problema.  En este proyecto, vamos a descubrir cómo crear un modelo generativo de texto, carácter por carácter, utilizando las redes neuronales recurrentes de LSTM en Python con Keras.


# **Descripción del problema: Generación de texto**

## **¿Qué utilidad tienen los modelos generativos?**
Estos modelos permiten en base a un conjunto de datos aprender y generar datos siguiendo secuencias aprendidas bajo los datos. Esto se utiliza mucho en **phising**. En base a mensajes preestablecidos, se generan emails, cuentas de usuarios, mensajes en aplicaciones de mensajería... etc.. para engañar al usuarios simulando que se encuentra en una conversión real. O también  en los **chatbots** para replicar la comunicación humana sin necesidad de tener a una persona al otro lado y facilitar la interacción por ejemplo con una empresa las 24 horas del día.

## **Enunciado**
Para implementar un modelo generativo sencillo vamos a seleccionar un libro de texto sencillo de nuestra infancia para utilizarlo como base de aprendizaje y a partir de ahí con Redes LSTM generar un modelo que pueda generar un texto a partir de su aprendizaje. Aprenderemos las dependencias entre los caracteres y las probabilidades condicionales de los personajes en las secuencias para que a su vez podamos generar secuencias de caracteres totalmente nuevas y originales.

Como datos de la práctica se entregan los siguientes ficheros:
- Cuento de Los 3 cerditos que sirva como base de aprendizaje (puedes utilizar el libro o texto que consideres)
- El proyecto realizado y explicado como ejemplo con una pequeña red neuronal


## **Desarrollo de una pequeña Red Neuronal LSTM Recurrente**
Empezaremos a preparar el conjunto de datos para el modelado. Nuestro libro de ejemplo no tiene encabezado o pie de pagina, pero en caso de que lo tenga deberemos eliminarlo.

Comenzaremos desarrollando una sencilla red LSTM para aprender secuencias de caracteres de Los tres cerditos. En este punto usaremos este modelo para generar nuevas secuencias de caracteres. Empecemos por importar las clases y funciones que pretendemos usar para entrenar a nuestro modelo.

In [ ]:
import sys
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

A continuación tendremos que cargar el texto ASCII del libro en la memoria y convertir todos los caracteres a minúsculas para reducir el vocabulario que la red debe aprender.

In [ ]:
#Cargamos el texto y lo pasamos a minuscula
filename = "los3.txt";
raw_text = open(filename).read()
raw_text = raw_text.lower()

Ahora que el libro está cargado, debemos preparar los datos para el modelado de la red neuronal. No podemos modelar los caracteres directamente, sino que debemos convertirlos en enteros. Podemos hacer esto fácilmente creando primero un conjunto de todos los caracteres distintos en el libro, y luego creando un mapa de cada carácter con un número entero único.

In [ ]:
#Crear mapeo de caracteres únicos a enteros, y un mapeo inverso
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

Por ejemplo, la lista de caracteres únicos en minúsculas ordenados en el libro es la siguiente:  
  
['\n', '\r', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', ':', ';', '?', '[', ']',
'_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xbb', '\xbf', '\xef'].  
  
Podemos ver que puede haber algunos caracteres que podríamos eliminar para limpiar aún más el conjunto de datos reduciendo el vocabulario y mejorando el proceso de modelado. Ahora que el libro ha sido cargado y el mapeo preparado, podemos resumir el conjunto de datos.

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total de caracteres:", n_chars)
print("Total vocales:", n_vocab)

Total de caracteres: 3229
Total vocales: 39


Podemos ver que el libro tiene algo menos de 3229 caracteres y que cuando se convierte a minúsculas sólo hay 39 caracteres distintos en el vocabulario para que la red aprenda. Mucho más que los 27 del alfabeto. Ahora tenemos que analizar los datos de formación para la red. Hay mucha flexibilidad en la forma que se decide dividir el texto y exponerlo a la red durante la formación. Aquí lo dividiremos en las siguientes secciones con una longitud fija de 100 caracteres, una longitud arbitraria. Podríamos fácilmente dividir los datos por frases y rellenar las secuencias más cortas y truncar las más largas.

Cada patrón de formación de la red se compone de 100 pasos de tiempo de un carácter (X) seguido de la salida de un carácter (y). Al crear estas secuencias, deslizamos esta ventana a lo largo de todo el libro, un carácter a la vez, lo que permite que cada carácter tenga la oportunidad de aprender de los 100 caracteres que lo precedieron (excepto los primeros 100 caracteres, por supuesto). Por ejemplo, si la longitud de la secuencia es 5 (por simplicidad) entonces los dos primeros patrones de entrenamiento serían los siguientes:

CHAPT -> E  
HAPTE -> R

A medida que dividimos el libro en estas secuencias, convertimos los caracteres en enteros usando nuestra tabla de búsqueda que preparamos anteriormente.

In [ ]:
#Preparar el conjunto de datos de entrada para los pares de salida codificados como enteros
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patrones: ", n_patterns)

Total patrones:  3129


La ejecución del código hasta este punto nos muestra que cuando dividimos el conjunto de datos en datos de formación para que la red se entere de que tenemos algo menos de Total Patterns: 3129 patrones de formación. Esto tiene sentido ya que excluyendo los primeros 100 caracteres, tenemos un patrón de entrenamiento para predecir cada uno de los caracteres restantes.

  
Ahora que hemos preparado nuestros datos de entrenamiento, necesitamos transformarlos para que sean adecuados para su uso con Keras. Primero debemos transformar la lista de secuencias de entrada en la forma[muestras, pasos de tiempo, características] esperada por una red LSTM. A continuación necesitamos reescalar los números enteros al rango 0-a-1 para hacer que los patrones sean más fáciles de aprender por la red LSTM que utiliza la función de activación sigmoide por defecto.  
  
Finalmente, necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación en caliente. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 47 caracteres diferentes en el vocabulario (una representación más fácil) en lugar de tratar de forzarlo a predecir con precisión el siguiente carácter. Cada valor de y se convierte en un vector disperso con una longitud de 38, lleno de ceros excepto con un 1 en la columna para la letra (entero) que el patrón representa. Por ejemplo, cuando n (valor entero 31) es un valor codificado en caliente, se ve como como sigue:

[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  
0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.  
0. 0. 0. 0. 0. 0. 0. 0.]  
  
Podemos implementar estos pasos como:

In [ ]:
#Remodelar X para que sea [muestras, pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
#Normalizacion
X = X / float(n_vocab)
#Codificacion en caliente con la variable de salida
y = np_utils.to_categorical(dataY)

Ahora podemos definir nuestro modelo LSTM. Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red utiliza el abandono con una probabilidad del 20%. La capa de salida es una capa densa que utiliza la función de activación de softmax para generar una predicción de probabilidad para cada uno de los 38 caracteres entre 0 y 1. El problema es en realidad un problema de clasificación de un solo carácter con 38 clases y, como tal, se define como la optimización del pérdida de registro (entropía cruzada), utilizando el algoritmo de optimización de ADAM para la velocidad.

In [ ]:
#Define el LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

No existe un conjunto de datos de prueba. Estamos modelando todo el conjunto de datos de entrenamiento para aprender la probabilidad de cada carácter en una secuencia. No nos interesa el modelo más preciso (precisión de clasificación) del conjunto de datos de formación. Este sería un modelo que predice cada carácter en el conjunto de datos de entrenamiento perfectamente. En su lugar, estamos interesados en una generalización del conjunto de datos que minimice la función de pérdida elegida. Buscamos un equilibrio entre la generalización y la adaptación, pero sin memorizar.
La red tarda en entrenarse (unos 35 segundos por epoch en mi pc usando la CPU). Dada la lentitud y debido a nuestros requisitos de optimización, utilizaremos el modelo de checkpointing para registrar todos los pesos de la red cada vez que se observe una mejora en las pérdidas en el fin de la epoch. Usaremos el mejor conjunto de pesos (la pérdida más baja) para instanciar nuestro sistema generativo en el siguiente punto.

In [ ]:
#Se define el checkpoint
filepath="pesos-los3-30-{epoch:02d}-{loss:.4f}.hdf5";
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Ahora podemos ajustar nuestro modelo a los datos. Aquí utilizamos 50 epoch y un tamaño de lote grande de 128 patrones.

In [ ]:
#Ajuste del modelo
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Podéis ver resultados diferentes debido a la naturaleza estocástica del modelo, y porque la dificultad de la semilla aleatoria en los modelos LSTM en obtener resultados 100% reproducibles. Esto no es una preocupación para este modelo generativo. Después de ejecutar el ejemplo, deberíais tener un número de puntos de control de peso en el directorio local. Podríamos borrarlos todos excepto el que tenga el menor valor de pérdida. Por ejemplo, cuando ejecuté este ejemplo, debajo estaba el punto de control con la menor pérdida que logré.  
  
**pesos-los3-30-50-1.7172**  
  
La pérdida de la red disminuyó casi todas las epoch y esperó que la red pueda dejar de entrenar por muchas más epoch. En el siguiente punto veremos el uso de este modelo para generar nuevas secuencias de texto.

# Generación de texto con una Red LSTM
La generación de texto utilizando la red LSTM es relativamente sencilla. En primer lugar, se cargan los datos y se define la red exactamente de la misma manera que hemos visto en el punto anterior, excepto que los pesos de la red se cargan desde un punto de control y la red no necesita ser entrenada.

In [ ]:
#Carga de los pesos de la red
filename="pesos-los3-30-50-1.7172.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Además, al preparar el mapeo de caracteres únicos a enteros, también debemos crear un mapeo inverso que podamos usar para convertir los enteros de nuevo en caracteres para que podamos entender las predicciones.

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Por último, tenemos que hacer predicciones. La forma más sencilla de usar el modelo LSTM de Keras para hacer predicciones es comenzar primero con una secuencia de semillas como entrada, generar el siguiente carácter y luego actualizar la secuencia de semillas para añadir el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1.000 caracteres de longitud). Podemos elegir un patrón de entrada aleatorio como nuestra semilla y luego imprimir los caracteres generados a medida que los generamos.

In [ ]:
#Toma una semilla aleatoria
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
#Genera los carácteres
for i in range(1000):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nHecho.")

Semilla:
" ago. los cerditos no lo volvieron a ver.  el mayor de ellos regaã±ã³ a los otros dos por haber sido  "
ae rarri lo meroi do lobo feroz  le lobo  el lobo  n  â¡quiã©n teme al lobo feroz, al lobo, el lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo! -- â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡

Ejecutando este ejemplo primero se produce la semilla aleatoria seleccionada, luego cada carácter a medida que se genera. Por ejemplo, a continuación se muestran los resultados de una ejecución de este generador de texto. La semilla aleatoria fue:  
  
*" â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo! ""*  
  
El texto generado con la semilla aleatoria (limpiado para presentación) fue:  
  
*ago. los cerditos no lo volvieron a ver.  el mayor de ellos regaã±ã³ a los otros dos por haber sido  "
ae rarri lo meroi do lobo feroz  le lobo  el lobo  n  â¡quiã©n teme al lobo feroz, al lobo, el lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo! -- â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo  al lobo!  - â¡quiã©n teme al lobo feroz, al lobo feroz! de cabo
Hecho.*  
  
Podemos notar algunas observaciones sobre el texto generado.  
  
Por lo general, se ajusta al formato de línea observado en el texto original de menos de 80 caracteres antes de una nueva línea.
Algunas de las palabras en secuencia tienen sentido, pero muchas no lo tienen.  
El hecho de que este modelo del libro, basado en caracteres, produzca resultados como estos es muy impresionante. Da una idea de las capacidades de aprendizaje de las redes LSTM. Los resultados son no perfectos.  
  
En el siguiente punto analizaremos la mejora de la calidad de los resultados mediante el desarrollo de un una red LSTM mucho más grande.

# Red Neuronal Recurrente LSTM más grande
Ahora vamos ha hacer los mismo creando una red mucho más grande. Mantendremos el mismo número de unidades de memoria en 256, pero añadiremos una segunda capa.